# 프로젝트 : 얼굴을 인식하여 캐릭터 씌우기

## https://opencv-python.readthedocs.io/en/latest/
- OpenCV 추가 공부 내용

## Face Detection vs Face Recognition

- Face Detection (얼굴 감지) : Face Detection은 이미지 또는 비디오에서 사람의 얼굴을 식별하는 과정입니다. 즉, 주어진 이미지에서 얼굴이 존재하는 위치를 찾아내는 것이 목표입니다. 이 과정에서는 얼굴이 어디에 있는지를 알아내지만, 어떤 사람의 얼굴인지는 식별하지 않습니다.

- Face Recognition (얼굴 인식) : Face Recognition은 특정 사람의 얼굴을 인식하는 과정입니다. 이미지에서 얼굴을 감지한 후, 그 얼굴이 어떤 사람의 얼굴인지를 판별합니다. 이를 위해 먼저 얼굴 특징을 추출하고, 이 특징들을 데이터베이스에 저장된 얼굴 특징과 비교하여 가장 유사한 사람을 찾아냅니다.

## 패키지 설치하는법
import mediapipe as mp => 실행 하면 안된다 (패키지를 프롬프트(터미널)을 통해 다운받아야한다)
- pip install mediapipe
- pip install --user mediapipe
### <아나콘다 프롬프트에 들어가는법>
아나콘다 => environment => 동그라미안에 세모(재생버튼) => open terminal(=prompt)실행

https://mediapipe-studio.webapps.google.com/home (얼굴인식)

In [1]:
import cv2
import mediapipe as mp

# 얼굴을 찾고, 찾은 얼굴에 표시를 해주기 위한 변수 정의
mp_face_detection = mp.solutions.face_detection #얼굴 검출을 위한 face_detection 모듈을 사용
mp_drawing = mp.solutions.drawing_utils # 얼굴의 특징을 그리기 위한 drawing_utils 모듈을 사용

# 동영상 파일 열기
cap = cv2.VideoCapture('face_video.mp4')

with mp_face_detection.FaceDetection(model_selection=0, min_detection_confidence=0.5) as face_detection:
    while cap.isOpened():
        success, image = cap.read()
        if not success:
            break

        # To improve performance, optionally mark the image as not writeable to
        # pass by reference.
        image.flags.writeable = False
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        results = face_detection.process(image)

        # Draw the face detection annotations on the image.
        image.flags.writeable = False
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        if results.detections:
            for detection in results.detections:
                mp_drawing.draw_detection(image, detection)

        # Flip the image horizontally for a selfie-view display.
        cv2.imshow('MediaPipe Face Detection', cv2.resize(image, None, fx=0.5, fy=0.5))

        if cv2.waitKey(1) == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


### 동영상을 같은 폴더(work space)에 놓지않아 오류가 떴었다.
> 오류코드 => OpenCV: Couldn't read video stream from file "face_video.mp4"
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.

## 눈 코에 동그라미 그리기

In [24]:
import cv2
import mediapipe as mp

# 얼굴을 찾고, 찾은 얼굴에 표시를 해주기 위한 변수 정의
mp_face_detection = mp.solutions.face_detection # 얼굴 검출을 위한 face_detection 모듈을 사용
mp_drawing = mp.solutions.drawing_utils # 얼굴의 특징을 그리기 위한 drawing_utils 모듈을 사용

# 동영상 파일 열기
cap = cv2.VideoCapture('face_video.mp4')

with mp_face_detection.FaceDetection(model_selection=0, min_detection_confidence=0.7) as face_detection: # model_selection 0m이내 근거리 1이면 5m이내 거리, min_detection은 threshold(0.5 = 50% 정도)
    while cap.isOpened():
        success, image = cap.read()
        if not success:
            break

        # To improve performance, optionally mark the image as not writeable to
        # pass by reference.
        image.flags.writeable = False
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        results = face_detection.process(image)

        # Draw the face detection annotations on the image.
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        if results.detections:
            # 6개 특징 : 오른쪽 눈, 왼쪽 눈 , 코 끝부분, 입 중심, 오른쪽 귀, 왼쪽 귀 (귀구슬점, 이주)
            for detection in results.detections:
                # mp_drawing.draw_detection(image, detection) # 빨간 점과 흰색 네모로 탐지알려주는건데 지저분해서 없앴다
                # print(detection)
                
                # 특정 위치 가져오기
                keypoints = detection.location_data.relative_keypoints
                right_eye = keypoints[0] # 오른쪽 눈
                left_eye = keypoints[1] # 오른쪽 눈
                nose_tip = keypoints[2]
                
                h, w, _ = image.shape # height, width, channel : 이미지로부터 세로, 가로 크기 가져옴
                right_eye = (int(right_eye.x * w), int(right_eye.y * h)) # 이미지 내에서 실제 좌표 (x, y)
                left_eye = (int(left_eye.x * w), int(left_eye.y * h))
                nose_tip = (int(nose_tip.x * w), int(nose_tip.y * h))
                
                # 양 눈에 동그라미 그리기
                cv2.circle(image, right_eye, 50, (255, 0, 0), 10, cv2.LINE_AA) # 파란색
                cv2.circle(image, left_eye, 50, (0, 255, 0), 10, cv2.LINE_AA) # 초록색
                # 코에 동그라미 그리기
                cv2.circle(image, nose_tip, 75, (0, 255, 255), 10, cv2.LINE_AA)
                

        # Flip the image horizontally for a selfie-view display.
        cv2.imshow('MediaPipe Face Detection', cv2.resize(image, None, fx=0.5, fy=0.5))

        if cv2.waitKey(1) == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

## 얼굴에 동물 캐릭터 씌워보기

In [34]:
import cv2
import mediapipe as mp

# 얼굴을 찾고, 찾은 얼굴에 표시를 해주기 위한 변수 정의
mp_face_detection = mp.solutions.face_detection # 얼굴 검출을 위한 face_detection 모듈을 사용
mp_drawing = mp.solutions.drawing_utils # 얼굴의 특징을 그리기 위한 drawing_utils 모듈을 사용

# 동영상 파일 열기
cap = cv2.VideoCapture('face_video.mp4')

# 이미지 불러오기
image_right_eye = cv2.imread('right_eye.png') # 100 x 100
image_left_eye = cv2.imread('left_eye.png') # 100 x 100
image_nose = cv2.imread('nose.png') # 300 x 100 (가로, 세로)

with mp_face_detection.FaceDetection(model_selection=0, min_detection_confidence=0.7) as face_detection: # model_selection 0m이내 근거리 1이면 5m이내 거리, min_detection은 threshold(0.5 = 50% 정도)
    while cap.isOpened():
        success, image = cap.read()
        if not success:
            break

        # To improve performance, optionally mark the image as not writeable to
        # pass by reference.
        image.flags.writeable = False
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        results = face_detection.process(image)

        # Draw the face detection annotations on the image.
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        if results.detections:
            # 6개 특징 : 오른쪽 눈, 왼쪽 눈 , 코 끝부분, 입 중심, 오른쪽 귀, 왼쪽 귀 (귀구슬점, 이주)
            for detection in results.detections:
                # mp_drawing.draw_detection(image, detection) # 빨간 점과 흰색 네모로 탐지알려주는건데 지저분해서 없앴다
                # print(detection)
                
                # 특정 위치 가져오기
                keypoints = detection.location_data.relative_keypoints
                right_eye = keypoints[0] # 오른쪽 눈
                left_eye = keypoints[1] # 오른쪽 눈
                nose_tip = keypoints[2]
                
                h, w, _ = image.shape # height, width, channel : 이미지로부터 세로, 가로 크기 가져옴
                right_eye = (int(right_eye.x * w) - 20, int(right_eye.y * h) - 100) # 이미지 내에서 실제 좌표 (x, y)
                left_eye = (int(left_eye.x * w) + 20, int(left_eye.y * h) - 100)
                nose_tip = (int(nose_tip.x * w), int(nose_tip.y * h))
                
                # 양 눈에 동그라미 그리기
                # cv2.circle(image, right_eye, 50, (255, 0, 0), 10, cv2.LINE_AA) # 파란색
                # cv2.circle(image, left_eye, 50, (0, 255, 0), 10, cv2.LINE_AA) # 초록색
                # 코에 동그라미 그리기
                # cv2.circle(image, nose_tip, 75, (0, 255, 255), 10, cv2.LINE_AA)
                
                # 각 특징에다가 이미지 그리기
                image[right_eye[1]-50:right_eye[1]+50, right_eye[0]-50:right_eye[0]+50] = image_right_eye
                image[left_eye[1]-50:left_eye[1]+50, left_eye[0]-50:left_eye[0]+50] = image_left_eye
                image[nose_tip[1]-50:nose_tip[1]+50, nose_tip[0]-150:nose_tip[0]+150] = image_nose
        
        # Flip the image horizontally for a selfie-view display.
        cv2.imshow('MediaPipe Face Detection', cv2.resize(image, None, fx=0.5, fy=0.5))

        if cv2.waitKey(1) == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

ValueError: could not broadcast input array from shape (300,300,3) into shape (100,300,3)

## 투명 이미지로 더 완성도 높여보기
- 이미지 크기조정 하여 저장하기
- 이미지 투명화 하는법 이미지 편집 프로그램 들어가서 마법봉 누르고 delete

In [28]:
import cv2
import mediapipe as mp

def overlay(image, x, y, w, h, overlay_image): # 이미지를 넣는데 동영상은 (100, 100, 3)으로 3채널인데 넣는 이미지는 (100, 100, 4) 체널로 맞지않아 변환해주는 함수
# 대상 이미지, x, y좌표, width, height, 덮어씌울 이미지
    alpha = overlay_image[:, :, 3] # BGRA
    mask_image = alpha / 255 # 0 ~ 255 -> 255로 나누면 0 ~ 1 사이의 값 (1 : 불투명, 0 : 완전투명)
    
    for c in range(0, 3): # channel BGR
        image[y-h:y+h, x-w:x+w, c] = (overlay_image[:, :, c] * mask_image) + (image[y-h:y+h, x-w:x+w, c] * (1 - mask_image))
        # (255, 255) -> (1,1)
        # (255, x) -> (1,0.3) (예시임 이코드에서는 0.3이 0 이고 때문에 0.7은 1이다)
        
        # 1 - mask_image ?
        # (0, 0)
        # (0, 0.7)
    
    
# 얼굴을 찾고, 찾은 얼굴에 표시를 해주기 위한 변수 정의
mp_face_detection = mp.solutions.face_detection # 얼굴 검출을 위한 face_detection 모듈을 사용
mp_drawing = mp.solutions.drawing_utils # 얼굴의 특징을 그리기 위한 drawing_utils 모듈을 사용

# 동영상 파일 열기
cap = cv2.VideoCapture('face_video.mp4')

# 이미지 불러오기
image_right_eye = cv2.imread('right_eye.png', cv2.IMREAD_UNCHANGED) # 100 x 100
image_left_eye = cv2.imread('left_eye.png', cv2.IMREAD_UNCHANGED) # 100 x 100
image_nose = cv2.imread('nose.png', cv2.IMREAD_UNCHANGED) # 300 x 100 (가로, 세로)

with mp_face_detection.FaceDetection(model_selection=0, min_detection_confidence=0.7) as face_detection: # model_selection 0m이내 근거리 1이면 5m이내 거리, min_detection은 threshold(0.5 = 50% 정도)
    while cap.isOpened():
        success, image = cap.read()
        if not success:
            break

        # To improve performance, optionally mark the image as not writeable to
        # pass by reference.
        image.flags.writeable = False
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        results = face_detection.process(image)

        # Draw the face detection annotations on the image.
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        if results.detections:
            # 6개 특징 : 오른쪽 눈, 왼쪽 눈 , 코 끝부분, 입 중심, 오른쪽 귀, 왼쪽 귀 (귀구슬점, 이주)
            for detection in results.detections:
                # mp_drawing.draw_detection(image, detection) # 빨간 점과 흰색 네모로 탐지알려주는건데 지저분해서 없앴다
                # print(detection)
                
                # 특정 위치 가져오기
                keypoints = detection.location_data.relative_keypoints
                right_eye = keypoints[0] # 오른쪽 눈
                left_eye = keypoints[1] # 오른쪽 눈
                nose_tip = keypoints[2]
                
                h, w, _ = image.shape # height, width, channel : 이미지로부터 세로, 가로 크기 가져옴
                right_eye = (int(right_eye.x * w) - 20, int(right_eye.y * h) - 100) # 이미지 내에서 실제 좌표 (x, y)
                left_eye = (int(left_eye.x * w) + 20, int(left_eye.y * h) - 100)
                nose_tip = (int(nose_tip.x * w), int(nose_tip.y * h))
                
                # 양 눈에 동그라미 그리기
                # cv2.circle(image, right_eye, 50, (255, 0, 0), 10, cv2.LINE_AA) # 파란색
                # cv2.circle(image, left_eye, 50, (0, 255, 0), 10, cv2.LINE_AA) # 초록색
                # 코에 동그라미 그리기
                # cv2.circle(image, nose_tip, 75, (0, 255, 255), 10, cv2.LINE_AA)
                
                # 각 특징에다가 이미지 그리기
                #image[right_eye[1]-50:right_eye[1]+50, right_eye[0]-50:right_eye[0]+50] = image_right_eye
                #image[left_eye[1]-50:left_eye[1]+50, left_eye[0]-50:left_eye[0]+50] = image_left_eye
                #image[nose_tip[1]-50:nose_tip[1]+50, nose_tip[0]-150:nose_tip[0]+150] = image_nose
                
                # image, x, y, w, h, overlay_image (overlay 함수 참고하기위해)
                overlay(image, *right_eye, 50, 50, image_right_eye)
                overlay(image, *left_eye, 50, 50, image_left_eye)
                overlay(image, *nose_tip, 150, 50, image_nose)
        
        # Flip the image horizontally for a selfie-view display.
        cv2.imshow('MediaPipe Face Detection', cv2.resize(image, None, fx=0.5, fy=0.5))

        if cv2.waitKey(1) == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

ValueError: operands could not be broadcast together with shapes (100,300) (300,300) 

In [35]:
# 불러올 이미지 확인하는 코드
image_right_eye = cv2.imread('nose.png', cv2.IMREAD_UNCHANGED)
image_right_eye.shape

(300, 300, 4)

## 투명이미지 씌우는게 계속 오류떠서 이미지 확인하고자 기본이미지에서 내가 코 300x300으로만 바꾸고 실행시켜봤다

In [39]:
import cv2
import mediapipe as mp

# 얼굴을 찾고, 찾은 얼굴에 표시를 해주기 위한 변수 정의
mp_face_detection = mp.solutions.face_detection # 얼굴 검출을 위한 face_detection 모듈을 사용
mp_drawing = mp.solutions.drawing_utils # 얼굴의 특징을 그리기 위한 drawing_utils 모듈을 사용

# 동영상 파일 열기
cap = cv2.VideoCapture('face_video.mp4')

# 이미지 불러오기
image_right_eye = cv2.imread('right_eye.png') # 100 x 100
image_left_eye = cv2.imread('left_eye.png') # 100 x 100
image_nose = cv2.imread('nose.png') # 300 x 300 (가로, 세로)

with mp_face_detection.FaceDetection(model_selection=0, min_detection_confidence=0.7) as face_detection: # model_selection 0m이내 근거리 1이면 5m이내 거리, min_detection은 threshold(0.5 = 50% 정도)
    while cap.isOpened():
        success, image = cap.read()
        if not success:
            break

        # To improve performance, optionally mark the image as not writeable to
        # pass by reference.
        image.flags.writeable = False
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        results = face_detection.process(image)

        # Draw the face detection annotations on the image.
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        if results.detections:
            # 6개 특징 : 오른쪽 눈, 왼쪽 눈 , 코 끝부분, 입 중심, 오른쪽 귀, 왼쪽 귀 (귀구슬점, 이주)
            for detection in results.detections:
                # mp_drawing.draw_detection(image, detection) # 빨간 점과 흰색 네모로 탐지알려주는건데 지저분해서 없앴다
                # print(detection)
                
                # 특정 위치 가져오기
                keypoints = detection.location_data.relative_keypoints
                right_eye = keypoints[0] # 오른쪽 눈
                left_eye = keypoints[1] # 오른쪽 눈
                nose_tip = keypoints[2]
                
                h, w, _ = image.shape # height, width, channel : 이미지로부터 세로, 가로 크기 가져옴
                right_eye = (int(right_eye.x * w) - 20, int(right_eye.y * h) - 100) # 이미지 내에서 실제 좌표 (x, y)
                left_eye = (int(left_eye.x * w) + 20, int(left_eye.y * h) - 100)
                nose_tip = (int(nose_tip.x * w), int(nose_tip.y * h))
                
                # 양 눈에 동그라미 그리기
                # cv2.circle(image, right_eye, 50, (255, 0, 0), 10, cv2.LINE_AA) # 파란색
                # cv2.circle(image, left_eye, 50, (0, 255, 0), 10, cv2.LINE_AA) # 초록색
                # 코에 동그라미 그리기
                # cv2.circle(image, nose_tip, 75, (0, 255, 255), 10, cv2.LINE_AA)
                
                # 각 특징에다가 이미지 그리기
                image[right_eye[1]-50:right_eye[1]+50, right_eye[0]-50:right_eye[0]+50] = image_right_eye
                image[left_eye[1]-50:left_eye[1]+50, left_eye[0]-50:left_eye[0]+50] = image_left_eye
                image[nose_tip[1]-150:nose_tip[1]+150, nose_tip[0]-150:nose_tip[0]+150] = image_nose
        
        # Flip the image horizontally for a selfie-view display.
        cv2.imshow('MediaPipe Face Detection', cv2.resize(image, None, fx=0.5, fy=0.5))

        if cv2.waitKey(1) == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

## 그리고 다시 강의보며 바로위 코드로 코드짜보자( 먼저 원래 오류떴던코드로 검사해보며 가의봐보자 시간아끼려고) 다시보면서 하기전에 가까스로 해결함

In [40]:
import cv2
import mediapipe as mp

def overlay(image, x, y, w, h, overlay_image):
    overlay_image = cv2.resize(overlay_image, (2*w, 2*h)) # 추가: overlay_image의 크기를 조정
    alpha = overlay_image[:, :, 3] / 255.0
    mask_image = alpha

    for c in range(0, 3):
        image[y-h:y+h, x-w:x+w, c] = (overlay_image[:, :, c] * mask_image) + (image[y-h:y+h, x-w:x+w, c] * (1 - mask_image))

mp_face_detection = mp.solutions.face_detection
mp_drawing = mp.solutions.drawing_utils

cap = cv2.VideoCapture('face_video.mp4')

image_right_eye = cv2.imread('right_eye.png', cv2.IMREAD_UNCHANGED)
image_left_eye = cv2.imread('left_eye.png', cv2.IMREAD_UNCHANGED)
image_nose = cv2.imread('nose.png', cv2.IMREAD_UNCHANGED)

with mp_face_detection.FaceDetection(model_selection=0, min_detection_confidence=0.7) as face_detection:
    while cap.isOpened():
        success, image = cap.read()
        if not success:
            break

        image.flags.writeable = False
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        results = face_detection.process(image)

        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        if results.detections:

            for detection in results.detections:

                keypoints = detection.location_data.relative_keypoints
                right_eye = keypoints[0]
                left_eye = keypoints[1]
                nose_tip = keypoints[2]
                
                h, w, _ = image.shape
                right_eye = (int(right_eye.x * w) - 20, int(right_eye.y * h) - 100)
                left_eye = (int(left_eye.x * w) + 20, int(left_eye.y * h) - 100)
                nose_tip = (int(nose_tip.x * w), int(nose_tip.y * h))

                overlay(image, *right_eye, 50, 50, image_right_eye)
                overlay(image, *left_eye, 50, 50, image_left_eye)
                overlay(image, *nose_tip, 100, 100, image_nose)
        
        cv2.imshow('MediaPipe Face Detection', cv2.resize(image, None, fx=0.5, fy=0.5))

        if cv2.waitKey(1) == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

## 코이미지의 크기와 상관없이 overlay에서 설정하는대로 코이미지의 크기가 자유롭게 바뀌며 코드가 정상적으로 실행된다

# 한참은 해결하지 못한 오류들

- 결국은 첫번째 오류는 이미지가 투명화 되어있지않았던것이고 (채널이 맞춰지지않음)
- 두번쨰 오류는 이미지 크기가 맞지않아서 오류가 생겼던것이다
- 그리고 주피터 노트북은 독자적이어서 데스크탑폴더에 저장하거나 비주얼스튜디오와 엮이면 오류가 생기고 파일경로가 엉킨다

In [14]:
import cv2
import mediapipe as mp

def overlay(image, x, y, w, h, overlay_image):
    overlay_image = cv2.resize(overlay_image, (2*w, 2*h)) # 추가: overlay_image의 크기를 조정
    alpha = overlay_image[:, :, 3] / 255.0
    mask_image = alpha

    for c in range(0, 3):
        image[y-h:y+h, x-w:x+w, c] = (overlay_image[:, :, c] * mask_image) + (image[y-h:y+h, x-w:x+w, c] * (1 - mask_image))

mp_face_detection = mp.solutions.face_detection
mp_drawing = mp.solutions.drawing_utils

cap = cv2.VideoCapture('face.mp4')

image_right_eye = cv2.imread('right_eye.png', cv2.IMREAD_UNCHANGED)
image_left_eye = cv2.imread('left_eye.png', cv2.IMREAD_UNCHANGED)
image_nose = cv2.imread('nose.png', cv2.IMREAD_UNCHANGED)

with mp_face_detection.FaceDetection(model_selection=0, min_detection_confidence=0.7) as face_detection:
    while cap.isOpened():
        success, image = cap.read()
        if not success:
            break

        image.flags.writeable = False
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        results = face_detection.process(image)

        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        if results.detections:

            for detection in results.detections:

                keypoints = detection.location_data.relative_keypoints
                right_eye = keypoints[0]
                left_eye = keypoints[1]
                nose_tip = keypoints[2]
                
                h, w, _ = image.shape
                right_eye = (int(right_eye.x * w) - 20, int(right_eye.y * h) - 100)
                left_eye = (int(left_eye.x * w) + 20, int(left_eye.y * h) - 100)
                nose_tip = (int(nose_tip.x * w), int(nose_tip.y * h))

                overlay(image, *right_eye, 50, 50, image_right_eye)
                overlay(image, *left_eye, 50, 50, image_left_eye)
                overlay(image, *nose_tip, 65, 65, image_nose)
        
        cv2.imshow('MediaPipe Face Detection', cv2.resize(image, None, fx=0.5, fy=0.5))

        if cv2.waitKey(25) == ord('q'):
            break

# 코덱 정의
fourcc = cv2.VideoWriter_fourcc(*'DIVX') # *로 여러인자로 떨어트려서 반환해준다. 'D','I'...이런식으로

# 프레임 크기, FPS
width = round(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = round(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS) * 1 #영상 재생 속도가 2배

out = cv2.VideoWriter('yoon.avi', fourcc, fps, (width, height))
# 저장 파일명, 코덱, FPS, 크기 (width, height)

while cap.isOpened():
    ret, frame = cap.read()
    
    if not ret:
        break
    
    out.write(frame) #영상 데이터만 저장 (소리 X)
    cv2.imshow('face', frame)
    if cv2.waitKey(1) == ord('q'):
        break

out.release() # 자원 해제


cap.release()
cv2.destroyAllWindows()

## 동영상 저장하는것까지 해보았다

In [1]:
import cv2
import mediapipe as mp

def overlay(image, x, y, w, h, overlay_image):
    overlay_image = cv2.resize(overlay_image, (2*w, 2*h))
    alpha = overlay_image[:, :, 3] / 255.0
    mask_image = alpha

    for c in range(0, 3):
        image[y-h:y+h, x-w:x+w, c] = (overlay_image[:, :, c] * mask_image) + (image[y-h:y+h, x-w:x+w, c] * (1 - mask_image))

mp_face_detection = mp.solutions.face_detection
mp_drawing = mp.solutions.drawing_utils

cap = cv2.VideoCapture('face.mp4')

image_right_eye = cv2.imread('right_eye.png', cv2.IMREAD_UNCHANGED)
image_left_eye = cv2.imread('left_eye.png', cv2.IMREAD_UNCHANGED)
image_nose = cv2.imread('nose.png', cv2.IMREAD_UNCHANGED)

# 코덱 정의
fourcc = cv2.VideoWriter_fourcc(*'DIVX')

# 프레임 크기, FPS
width = round(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = round(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)

out = cv2.VideoWriter('yoon.avi', fourcc, fps, (width, height)) # 저장 파일명, 코덱, FPS, 크기 (width, height)

with mp_face_detection.FaceDetection(model_selection=0, min_detection_confidence=0.7) as face_detection:
    while cap.isOpened():
        success, image = cap.read()
        if not success:
            break

        image.flags.writeable = False
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        results = face_detection.process(image)

        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        if results.detections:
            for detection in results.detections:
                keypoints = detection.location_data.relative_keypoints
                right_eye = keypoints[0]
                left_eye = keypoints[1]
                nose_tip = keypoints[2]
                
                h, w, _ = image.shape
                right_eye = (int(right_eye.x * w) - 20, int(right_eye.y * h) - 100)
                left_eye = (int(left_eye.x * w) + 20, int(left_eye.y * h) - 100)
                nose_tip = (int(nose_tip.x * w), int(nose_tip.y * h))

                overlay(image, *right_eye, 50, 50, image_right_eye)
                overlay(image, *left_eye, 50, 50, image_left_eye)
                overlay(image, *nose_tip, 65, 65, image_nose)

        out.write(cv2.resize(image, (width, height)))  # 이미지 크기 조정 후 저장
        cv2.imshow('MediaPipe Face Detection', cv2.resize(image, None, fx=0.5, fy=0.5))

        if cv2.waitKey(25) == ord('q'):
            break

out.release() # 자원 해제
cap.release()
cv2.destroyAllWindows()


INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
